In [ ]:
!pip install -qU google-genai langchain langchain_community langchain-google-community langchain-google-genai google-api-python-client google-auth google-auth-oauthlib


In [ ]:
import os
from google import genai
from google.genai import types
from google.auth.transport.requests import Request
from google.oauth2 import service_account
from googleapiclient.discovery import build
from IPython.display import Markdown, display
from langchain.tools import tool
from langchain_google_community import GoogleSearchAPIWrapper


In [ ]:
# Set API key
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    GOOGLE_API_KEY = input("Please enter your Google API key: ")

if not GOOGLE_API_KEY:
    raise ValueError("Google API key not found.")

client = genai.Client(api_key=GOOGLE_API_KEY)
genai_model = "gemini-2.0-flash"

# Memory for conversation
conversation_history = []

# Google Search setup
os.environ["GOOGLE_CSE_ID"] = "GOOGLE_CSE_ID"
os.environ["GOOGLE_API_KEY"] = "GOOGLE_API_KEY"

search_wrapper = GoogleSearchAPIWrapper()


In [ ]:
# Load credentials from service account
SERVICE_ACCOUNT_FILE = "3.json"
SCOPES = ["https://www.googleapis.com/auth/calendar"]

creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES
)
calendar_service = build("calendar", "v3", credentials=creds)

@tool
def calendar_tool(query: str) -> str:
    """Creates a calendar event (currently fixed time)."""
    event = {
        "summary": "AI Scheduled Meeting",
        "start": {"dateTime": "2025-05-16T10:00:00+05:00"},
        "end": {"dateTime": "2025-05-16T11:00:00+05:00"},
    }
    try:
        created_event = calendar_service.events().insert(calendarId="primary", body=event).execute()
        return f"Calendar event created: {created_event.get('htmlLink')}"
    except Exception as e:
        return f"Error creating calendar event: {e}"


In [ ]:
def decide_tool(query: str) -> str:
    prompt = f"""You are a smart router. Classify the query into one of these tools:
1. 'calendar' if it's about scheduling
2. 'search' for looking up information
3. 'chat' for general conversation

Query: "{query}"
Answer with one word: calendar, search, or chat."""

    decision = client.models.generate_content(
        model=genai_model,
        contents=[types.Content(role="user", parts=[types.Part(text=prompt)])]
    )
    return decision.candidates[0].content.parts[0].text.strip().lower()


In [ ]:
print("Chatbot with memory, search & calendar started! Type 'quit' to exit.")

while True:
    user_input = input("You: ")
    if user_input.lower() == "quit":
        print("Goodbye!")
        break

    # Append user message to memory
    conversation_history.append(types.Content(role="user", parts=[types.Part(text=user_input)]))

    # Tool routing
    tool_choice = decide_tool(user_input)

    if tool_choice == "search":
        print("[🔍 Using Google Search]")
        search_result = search_wrapper.run(user_input)
        response_text = search_result
    elif tool_choice == "calendar":
        print("[📅 Using Google Calendar Tool]")
        response_text = calendar_tool.invoke(user_input)
    else:
        print("[💬 Chat Response]")
        response = client.models.generate_content(
            model=genai_model,
            contents=conversation_history
        )
        response_text = response.candidates[0].content.parts[0].text

    # Show and store model response
    display(Markdown(response_text))
    conversation_history.append(types.Content(role="model", parts=[types.Part(text=response_text)]))
